In [ ]:
!pip install tensorflow-hub
!pip install tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.0
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


In [ ]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2UU01P/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2UU01P/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(1)))
train_examples_batch

<tf.Tensor: shape=(1,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."],
      dtype=object)>

In [ ]:
train_labels_batch

<tf.Tensor: shape=(1,), dtype=int64, numpy=array([0])>

In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=False)

<tf.Tensor: shape=(0, 50), dtype=float32, numpy=array([], shape=(0, 50), dtype=float32)>

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(1)))
print(hub_layer(train_examples_batch))

tf.Tensor(
[[ 0.5423195  -0.0119017   0.06337538  0.06862972 -0.16776837 -0.10581174
   0.16865303 -0.04998824 -0.31148055  0.07910346  0.15442263  0.01488662
   0.03930153  0.19772711 -0.12215476 -0.04120981 -0.2704109  -0.21922152
   0.26517662 -0.80739075  0.25833532 -0.3100421   0.28683215  0.1943387
  -0.29036492  0.03862849 -0.7844411  -0.0479324   0.4110299  -0.36388892
  -0.58034706  0.30269456  0.3630897  -0.15227164 -0.44391504  0.19462997
   0.19528408  0.05666234  0.2890704  -0.28468323 -0.00531206  0.0571938
  -0.3201318  -0.04418665 -0.08550783 -0.55847436 -0.23336391 -0.20782952
  -0.03543064 -0.17533456]], shape=(1, 50), dtype=float32)


In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 833
Non-trainable params: 48,190,600
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 4s 77ms/step - loss: 0.6608 - accuracy: 0.5239 - val_loss: 0.6417 - val_accuracy: 0.5503
Epoch 2/10
30/30 [==============================] - 3s 69ms/step - loss: 0.6271 - accuracy: 0.5828 - val_loss: 0.6127 - val_accuracy: 0.6338
Epoch 3/10
30/30 [==============================] - 3s 66ms/step - loss: 0.6003 - accuracy: 0.6348 - val_loss: 0.5887 - val_accuracy: 0.6562
Epoch 4/10
30/30 [==============================] - 3s 69ms/step - loss: 0.5801 - accuracy: 0.6626 - val_loss: 0.5706 - val_accuracy: 0.6713
Epoch 5/10
30/30 [==============================] - 3s 68ms/step - loss: 0.5644 - accuracy: 0.6794 - val_loss: 0.5581 - val_accuracy: 0.6730
Epoch 6/10
30/30 [==============================] - 3s 72ms/step - loss: 0.5525 - accuracy: 0.6910 - val_loss: 0.5472 - val_accuracy: 0.6984
Epoch 7/10
30/30 [==============================] - 3s 71ms/step - loss: 0.5435 - accuracy: 0.7029 - val_loss: 0.5395 - val_accuracy: 0.7000
Epoch 8/10
30

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.5342 - accuracy: 0.7072 - 2s/epoch - 45ms/step
loss: 0.534
accuracy: 0.707


In [ ]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 50)                48190600  
                                                                 
 dense_2 (Dense)             (None, 16)                816       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 21s 647ms/step - loss: 0.6575 - accuracy: 0.5391 - val_loss: 0.5967 - val_accuracy: 0.6115
Epoch 2/10
30/30 [==============================] - 19s 628ms/step - loss: 0.5266 - accuracy: 0.7015 - val_loss: 0.4822 - val_accuracy: 0.7759
Epoch 3/10
30/30 [==============================] - 19s 628ms/step - loss: 0.3837 - accuracy: 0.8375 - val_loss: 0.3845 - val_accuracy: 0.8315
Epoch 4/10
30/30 [==============================] - 19s 627ms/step - loss: 0.2731 - accuracy: 0.8965 - val_loss: 0.3351 - val_accuracy: 0.8527
Epoch 5/10
30/30 [==============================] - 19s 628ms/step - loss: 0.1982 - accuracy: 0.9301 - val_loss: 0.3127 - val_accuracy: 0.8627
Epoch 6/10
30/30 [==============================] - 19s 627ms/step - loss: 0.1443 - accuracy: 0.9546 - val_loss: 0.3051 - val_accuracy: 0.8708
Epoch 7/10
30/30 [==============================] - 19s 632ms/step - loss: 0.1037 - accuracy: 0.9715 - val_loss: 0.3087 - val_accuracy: 0.8725

In [ ]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 5s - loss: 0.3697 - accuracy: 0.8509 - 5s/epoch - 96ms/step
loss: 0.370
accuracy: 0.851
